In [1]:
import pathlib
import pandas as pd
import numpy as np
import pandas_profiling
import warnings
warnings.filterwarnings('ignore')
from mltools.mlcommon import one_hot_dataframe

In [2]:
project_directory = pathlib.Path().resolve().parent
data_directory  = project_directory.joinpath('data/raw')
report_directory  = project_directory.joinpath('reports')

In [3]:
train_file = 'train.csv'
test_file  = 'test.csv'
report_file = 'profile_report.html'
train_filepath = data_directory.joinpath(train_file)
test_filepath = data_directory.joinpath(test_file)
report_filepath = report_directory.joinpath(report_file)

In [4]:
def get_df_describe(df):
    describe_df  = df.describe(include='all').T
    describe_df['Null_Count']= df.isnull().sum()
    return describe_df

In [5]:
def has_cabin(deck_level):
    if (deck_level =='X'):
        return 'No'
    else: 
        return 'Yes'


In [6]:
def tweak_dataset(df):
    df['FamilySize'] = df["SibSp"] +df["Parch"] + 1
    df['Embarked'].fillna('S' ,inplace =True)
    df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.',expand =False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Sir', 'Jonkheer', 'Dona','Don'], 'Other')
    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Dr', 'Major', 'Rev'],'Officer')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

    df['Cabin'].fillna('XXX' ,inplace=True)
    df['Deck_Level'] = df['Cabin'].apply(lambda x : x[0]) 
     
    df.drop(['Name','Ticket','Cabin',],inplace=True,axis =1) 
    #df = pd.get_dummies(df,drop_first=True)
    return df

In [7]:
def fill_missing_age(row):
    if pd.isnull(row['Age']):
        return median_ages[row['Sex'],row['Pclass'],row['Title'] ,row['SibSp'] ,row['Parch']]
    else:
        return row['Age']


In [8]:
train_df = pd.read_csv(train_filepath)
test_df = pd.read_csv(test_filepath)

In [9]:
pandas_profiling.ProfileReport(train_df).to_file(report_filepath)

In [10]:
#train_df = tweak_dataset(train_df)
#test_df = tweak_dataset(test_df)

In [11]:
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)

In [13]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [12]:
Check

NameError: name 'Check' is not defined

In [16]:
full_dataset =pd.concat(objs=[train_df, test_df], axis=0,sort=True).reset_index(drop=True)  

In [17]:
train_dataset =full_dataset[full_dataset['Survived'].notnull()]
train_y = full_dataset[full_dataset['Survived'].notnull()]['Survived']
test_dataset =full_dataset[full_dataset['Survived'].isnull()]
#train_dataset["Survived"] = train_dataset["Survived"].astype("int")
test_passengers = test_dataset['PassengerId']
train_dataset.drop(labels=["Survived",'PassengerId'],axis = 1,inplace=True)
test_dataset.drop(labels=["Survived",'PassengerId'],axis = 1,inplace=True)

In [29]:
cate_features_index = np.where(train_dataset.dtypes != float)[0]

In [20]:
from sklearn.model_selection  import  train_test_split

In [21]:
X_train, X_val, y_train, y_val = train_test_split(train_dataset,train_y,test_size=0.3, random_state=42)

In [22]:
from sklearn import metrics

In [23]:
from catboost import CatBoostClassifier, Pool, cv

In [34]:
catboost_model = CatBoostClassifier(loss_function='Logloss', custom_metric='AUC:hints=skip_train~false', metric_period=20)

In [35]:
catboost_model.fit(X_train, y_train,eval_set=(X_val, y_val),cat_features=cate_features_index)

Learning rate set to 0.074246
0:	learn: 0.6527757	test: 0.6524415	best: 0.6524415 (0)	total: 35ms	remaining: 35s
20:	learn: 0.4286319	test: 0.4426418	best: 0.4426418 (20)	total: 1.21s	remaining: 56.2s
40:	learn: 0.4006740	test: 0.4224243	best: 0.4224243 (40)	total: 2.27s	remaining: 53.2s
60:	learn: 0.3908619	test: 0.4187190	best: 0.4187190 (60)	total: 3.09s	remaining: 47.6s
80:	learn: 0.3795637	test: 0.4143556	best: 0.4143556 (80)	total: 3.92s	remaining: 44.5s
100:	learn: 0.3655047	test: 0.4084590	best: 0.4084590 (100)	total: 4.86s	remaining: 43.3s
120:	learn: 0.3516161	test: 0.4062774	best: 0.4062774 (120)	total: 6.17s	remaining: 44.8s
140:	learn: 0.3340691	test: 0.4043454	best: 0.4043454 (140)	total: 7.63s	remaining: 46.5s
160:	learn: 0.3185344	test: 0.4038973	best: 0.4038973 (160)	total: 9.15s	remaining: 47.7s
180:	learn: 0.3021198	test: 0.4001843	best: 0.4001843 (180)	total: 10.7s	remaining: 48.3s
200:	learn: 0.2884311	test: 0.4040137	best: 0.4001843 (180)	total: 12.1s	remaining: 4

In [36]:
cv_data = cv(Pool(X_train,y_train,cat_features=cate_features_index),catboost_model.get_params(),fold_count=10) 

0:	learn: 0.6766921	test: 0.6771108	best: 0.6771108 (0)	total: 4.07s	remaining: 1h 7m 48s
20:	learn: 0.5024130	test: 0.5103828	best: 0.5103828 (20)	total: 1m 9s	remaining: 54m 9s
40:	learn: 0.4547821	test: 0.4699913	best: 0.4699913 (40)	total: 2m 13s	remaining: 51m 57s
60:	learn: 0.4325228	test: 0.4527157	best: 0.4527157 (60)	total: 3m 16s	remaining: 50m 19s
80:	learn: 0.4188960	test: 0.4427315	best: 0.4427315 (80)	total: 4m 19s	remaining: 49m 6s
100:	learn: 0.4084186	test: 0.4354614	best: 0.4354614 (100)	total: 5m 21s	remaining: 47m 40s
120:	learn: 0.4020894	test: 0.4326019	best: 0.4326019 (120)	total: 6m 24s	remaining: 46m 32s
140:	learn: 0.3967853	test: 0.4304271	best: 0.4304271 (140)	total: 7m 26s	remaining: 45m 17s
160:	learn: 0.3906035	test: 0.4279699	best: 0.4279699 (160)	total: 8m 31s	remaining: 44m 23s
180:	learn: 0.3857642	test: 0.4255557	best: 0.4255557 (180)	total: 9m 33s	remaining: 43m 13s
200:	learn: 0.3816629	test: 0.4247608	best: 0.4247608 (200)	total: 10m 33s	remaining

In [37]:
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score,recall_score,precision_score

def model_evalution(model,x_train,y_train,x_test,y_test):
    print("####################### model Evalution started #######################")
    train_pre = model.predict(x_train)
    test_pre = model.predict(x_test)
    train_pro = model.predict_proba(x_train)
    test_pro = model.predict_proba(x_test)

    print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
    print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
    print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
    print("Train F1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
    print("Train recall: {0} \t Test recall: {1}".format(recall_score(y_train, train_pre),recall_score(y_test,test_pre)))
    print("Train precision: {0} \t Test Precision: {1}".format(precision_score(y_train, train_pre),precision_score(y_test,test_pre)))
    print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score

In [38]:
model_evalution(catboost_model,X_train, y_train,X_val,y_val)

####################### model Evalution started #######################
Train Accuracy: 0.9229534510433387 	 Test Accuracy: 0.8097014925373134
Train Loss: 0.07704654895666131 	 Test Loss: 0.19029850746268656
Train AUC: 0.9845337485643608 	 Test AUC: 0.8954495897171055
Train F1: 0.8938053097345133 	 Test F1: 0.7487684729064039
Train recall: 0.8744588744588745 	 Test recall: 0.6846846846846847
Train precision: 0.9140271493212669 	 Test Precision: 0.8260869565217391
Train Confusion Matrix: 
[[373  19]
 [ 29 202]] 
 Test Confusion Matrix: 
[[141  16]
 [ 35  76]]


In [39]:
test_y =catboost_model.predict(test_dataset)

In [41]:
submission =pd.DataFrame({'PassengerId':test_passengers.values ,'Survived':test_y}) 
import datetime
FORMAT = '%Y%m%d%H%M%S'
timestamp=datetime.datetime.now().strftime(FORMAT)
filename ="Titanic_CatBoost"+timestamp+"_out.csv"
submission.to_csv(filename,index=False)

In [43]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
# Number of trees 
iterations = [int(x) for x in np.linspace(start = 100, stop = 500, num = 4)]
# Maximum number of levels in tree
depth = [int(x) for x in np.linspace(1, 10, num = 11)]
 
# Minimum number of samples required to split a node
l2_leaf_reg = [1,2,3,5,8]
# Minimum number of samples required at each leaf node
loss_function = ['Logloss','CrossEntropy']
# Method of selecting samples for training each tree
learning_rate =[0.005,0.03,0.1]
border_count =[32,]
ctr_border_count=[50,]
thread_count=4
eval_metric =["Precision"]
params = {'depth':depth,
          'iterations':iterations,
          'learning_rate':learning_rate, 
          'l2_leaf_reg':l2_leaf_reg,
          #'border_count':border_count,
          #'ctr_border_count':ctr_border_count,
          #'thread_count':thread_count,
          #'eval_metric':eval_metric,
         }
pprint(params)

{'depth': [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'iterations': [100, 233, 366, 500],
 'l2_leaf_reg': [1, 2, 3, 5, 8],
 'learning_rate': [0.005, 0.03, 0.1]}


In [44]:
from sklearn.model_selection import RandomizedSearchCV

In [45]:
cb_model = CatBoostClassifier()

randm = RandomizedSearchCV(estimator=cb_model, param_distributions = params, 
                               cv = 3, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="E67": Cannot convert 'b'E67'' to float

In [ ]:
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

In [ ]:
# catboost_model = CatBoostClassifier(iterations=400,
#                                     verbose=True,eval_metric="F1",
#                                     learning_rate=0.1,
#                                     class_weights=[1,3],
#                                     depth=3,l2_leaf_reg=3,bagging_temperature=0.5)

In [ ]:
catboost_model = randm.best_estimator_

In [ ]:
model_evalution(catboost_model,X_train, y_train,X_val,y_val)

In [ ]:
test_y =catboost_model.predict(test_dataset)

In [ ]:
submission =pd.DataFrame({'PassengerId':test_passengers.values ,'Survived':test_y}) 
#submission['Loan_Status'].replace(0, 'N',inplace=True)
#submission['Loan_Status'].replace(1, 'Y',inplace=True)
import datetime
FORMAT = '%Y%m%d%H%M%S'
timestamp=datetime.datetime.now().strftime(FORMAT)
filename ="Titanic_CatBoost"+timestamp+"_out.csv"
#submission.to_csv(filename,index=False)

submission.to_csv(filename,index=False)